In [48]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import category_encoders as ce
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

train_values = pd.read_csv('train_values.csv', sep=',')
train_labels = pd.read_csv('train_labels.csv', sep=',')
x_test = pd.read_csv('test_values.csv', sep=',')

X = train_values.copy()
X.drop('building_id', inplace=True, axis=1)
y = train_labels.copy()
y.drop('building_id', inplace=True, axis=1)
x_test_droped = x_test.copy()
x_test_droped.drop('building_id', inplace=True, axis=1)


In [1]:
!pip install xgboost

In [49]:
X[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]] = X[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]].astype(str)
x_test_droped[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]] = x_test_droped[["geo_level_1_id","geo_level_2_id","geo_level_3_id"]].astype(str)
print(type(X["geo_level_1_id"][0]))


<class 'str'>


In [50]:
first_ce = ce.OneHotEncoder(cols=["land_surface_condition","foundation_type","roof_type","ground_floor_type",
                                  "other_floor_type","position","legal_ownership_status"]) 

ce_te = ce.TargetEncoder(cols=["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"])
xx = X[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]]
x_test_target_encoded = x_test_droped[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]]
yy = y['damage_grade']

ce_te.fit(xx,yy)

X[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]] = ce_te.transform(xx)

x_test_droped[["geo_level_1_id","geo_level_2_id","geo_level_3_id","plan_configuration"]] = ce_te.transform(x_test_target_encoded)

#change geolocations to categorical and encode 
X = first_ce.fit_transform(X)
x_test_droped = first_ce.fit_transform(x_test_droped)

reg = LinearRegression().fit(X, y)
    
est = sm.OLS(y, X)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           damage_grade   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     4416.
Date:                Thu, 28 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:19:53   Log-Likelihood:            -1.5939e+05
No. Observations:              260601   AIC:                         3.189e+05
Df Residuals:                  260548   BIC:                         3.194e+05
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
geo_level_1_id                             0.0384      0.005      7.153      0.000       0.028       0.049
geo_level_2_id                            -0.0336      0.007     -5.152      0.000      -0.046      -0.021
geo_level_3_id                             0.9114      0.005    179.384      0.000       0.901       0.921
count_floors_pre_eq                        0.0012      0.002      0.527      0.598      -0.003       0.006
age                                        0.0001    1.2e-05      9.918      0.000    9.56e-05       0.000
area_percentage                           -0.0002      0.000     -0.928      0.354      -0.001       0.000
height_percentage                          0.0026      0.001      3.441      0.001       0.001       0.004
land_surface_condition_1                3.483e+08   1.27e+09      0.275      0.784   -2.14e+09    2.83e+09
land_surface_condition_2                3.483e+08   1.27e+09      0.275      0.784   -2.14e+09    2.83e+09
land_surface_condition_3                3.483e+08   1.27e+09      0.275      0.784   -2.14e+09    2.83e+09
foundation_type_1                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_2                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_3                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_4                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
foundation_type_5                       1.898e+08   6.91e+08      0.275      0.784   -1.16e+09    1.54e+09
roof_type_1                             3.099e+07   1.13e+08      0.275      0.784    -1.9e+08    2.52e+08
roof_type_2                             3.099e+07   1.13e+08      0.275      0.784    -1.9e+08    2.52e+08
roof_type_3                             3.099e+07   1.13e+08      0.275      0.784    -1.9e+08    2.52e+08
ground_floor_type_1                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_2                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_3                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_4                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
ground_floor_type_5                     1.319e+08    4.8e+08      0.275      0.784   -8.09e+08    1.07e+09
other_floor_type_1                     -7.529e+07   2.74e+08     -0.275      0.784   -6.13e+08    4.62e+08
other_floor_type_2                     -7.529e+07   2.74e+08     -0.275      0.784   -6.13e+08    4.62e+08
other_floor_type_3                     -7.529e+07   2.74e+08     -0.275      0.784   -6

,land_surface_condition_1,land_surface_condition_2,land_surface_condition_3
0,1,0,0
1,0,1,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
260596,0,0,1
260597,1,0,0
260598,1,0,0
260599,1,0,0


In [54]:
#dropped_columns_encoded_x_train = X.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])
#dropped_columns_encoded_x_test = x_test_droped.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other"])

dropped_columns_encoded_x_train = X.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other","legal_ownership_status_1","legal_ownership_status_2","legal_ownership_status_3","legal_ownership_status_4"])
dropped_columns_encoded_x_test = x_test_droped.drop(columns=["count_floors_pre_eq","has_secondary_use_health_post","has_secondary_use_gov_office","has_secondary_use_use_police","has_secondary_use_other","legal_ownership_status_1","legal_ownership_status_2","legal_ownership_status_3","legal_ownership_status_4"])
#Import Random Forest Model




In [56]:
#change geolocations to categorical and encode 
#X = first_ce.fit_transform(X)
#x_test_droped = first_ce.fit_transform(x_test_droped)
from sklearn.ensemble import RandomForestClassifier

reg = LinearRegression().fit(dropped_columns_encoded_x_train, y)
    
est = sm.OLS(y, dropped_columns_encoded_x_train)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           damage_grade   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.468
Method:                 Least Squares   F-statistic:                     5216.
Date:                Thu, 28 Jul 2022   Prob (F-statistic):               0.00
Time:                        16:28:40   Log-Likelihood:            -1.5942e+05
No. Observations:              260601   AIC:                         3.189e+05
Df Residuals:                  260556   BIC:                         3.194e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
geo_level_1_id                             0.0389      0.005      7.359      0.000       0.029       0.049
geo_level_2_id                            -0.0344      0.007     -5.278      0.000      -0.047      -0.022
geo_level_3_id                             0.9113      0.005    179.357      0.000       0.901       0.921
age                                        0.0001    1.2e-05      9.881      0.000    9.52e-05       0.000
area_percentage                           -0.0003      0.000     -1.172      0.241      -0.001       0.000
height_percentage                          0.0028      0.001      4.539      0.000       0.002       0.004
land_surface_condition_1               -7.575e+08   2.86e+09     -0.265      0.791   -6.35e+09    4.84e+09
land_surface_condition_2               -7.575e+08   2.86e+09     -0.265      0.791   -6.35e+09    4.84e+09
land_surface_condition_3               -7.575e+08   2.86e+09     -0.265      0.791   -6.35e+09    4.84e+09
foundation_type_1                       3.758e+08   1.42e+09      0.265      0.791    -2.4e+09    3.15e+09
foundation_type_2                       3.758e+08   1.42e+09      0.265      0.791    -2.4e+09    3.15e+09
foundation_type_3                       3.758e+08   1.42e+09      0.265      0.791    -2.4e+09    3.15e+09
foundation_type_4                       3.758e+08   1.42e+09      0.265      0.791    -2.4e+09    3.15e+09
foundation_type_5                       3.758e+08   1.42e+09      0.265      0.791    -2.4e+09    3.15e+09
roof_type_1                             2.685e+07   1.01e+08      0.265      0.791   -1.72e+08    2.25e+08
roof_type_2                             2.685e+07   1.01e+08      0.265      0.791   -1.72e+08    2.25e+08
roof_type_3                             2.685e+07   1.01e+08      0.265      0.791   -1.72e+08    2.25e+08
ground_floor_type_1                     2.165e+08   8.16e+08      0.265      0.791   -1.38e+09    1.82e+09
ground_floor_type_2                     2.165e+08   8.16e+08      0.265      0.791   -1.38e+09    1.82e+09
ground_floor_type_3                     2.165e+08   8.16e+08      0.265      0.791   -1.38e+09    1.82e+09
ground_floor_type_4                     2.165e+08   8.16e+08      0.265      0.791   -1.38e+09    1.82e+09
ground_floor_type_5                     2.165e+08   8.16e+08      0.265      0.791   -1.38e+09    1.82e+09
other_floor_type_1                     -1.048e+08   3.95e+08     -0.265      0.791   -8.79e+08    6.69e+08
other_floor_type_2                     -1.048e+08   3.95e+08     -0.265      0.791   -8.79e+08    6.69e+08
other_floor_type_3                     -1.048e+08   3.95e+08     -0.265      0.791   -8.79e+08    6.69e+08
other_floor_type_4                     -1.048e+08   3.95e+08     -0.265      0.791   -8

In [57]:
#Create a Gaussian Classifier
clf=RandomForestClassifier()
clf = clf.fit(dropped_columns_encoded_x_train,y)
y_pred=clf.predict(dropped_columns_encoded_x_test)
y_pred

array([3, 2, 3, ..., 2, 2, 2])

In [58]:
from sklearn.model_selection import cross_val_score
y_train_vector = y.values.flatten()
print(np.mean(cross_val_score(clf, dropped_columns_encoded_x_train, y_train_vector)))

0.7408605504020598


In [42]:
#Get submission ready for random forest attempt
import base64
from pathlib import Path

submission_random_forest = x_test["building_id"]
i = pd.DataFrame(np.array(y_pred))

submission_concatenated = pd.concat([submission_random_forest, i], ignore_index="true", axis=1)
submission_renamed_columns = submission_concatenated.rename(columns={0: "building_id", 1: "damage_grade"})
filepath = Path('./submission_random_forest_4.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
submission_renamed_columns.to_csv(filepath, index=False)


In [65]:
'''from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


class SampleModel(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator=None):
        self.estimator = estimator
    def fit(self, X, y):
        self.estimator.fit(X, y)
    def predict(self, X):
        pred = self.estimator.predict(X)
        return pred

param_grid = {
    "estimator": [
        #GradientBoostingClassifier(random_state=42),
        RandomForestClassifier(random_state=42)
        #,LogisticRegression(random_state=42, max_iter=5000)
    ]
}

model = SampleModel()

scoring = ["accuracy", "precision", "recall"]

gscv = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scoring,
    refit=False
)
gscv.fit(dropped_columns_encoded_x_train, y_train_vector)
results_df = pd.DataFrame(gscv.cv_results_)
print(results_df)'''

KeyboardInterrupt: 

In [3]:
import xgboost as xgb

# Show all messages, including ones pertaining to debugging
xgb.set_config(verbosity=2)

# Get current value of global configuration
# This is a dict containing all parameters in the global configuration,
# including 'verbosity'
config = xgb.get_config()
assert config['verbosity'] == 2

# Example of using the context manager xgb.config_context().
# The context manager will restore the previous value of the global
# configuration upon exiting.
with xgb.config_context(verbosity=0):
    # Suppress warning caused by model generated with XGBoost version < 1.0.0
    bst = xgb.Booster()
assert xgb.get_config()['verbosity'] == 2  # old value restored

xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

xgb_model.fit(dropped_columns_encoded_x_train, y)

y_pred = xgb_model.predict(dropped_columns_encoded_x_test)
y_pred

NameError: name 'dropped_columns_encoded_x_train' is not defined